In [4]:
    import os
    from snow_processing import prepare_modis_mask

    BASE_IN  = r"F:\GeoinformaticsProject\Data\MODIS_SnowMask"
    BASE_OUT = r"F:\GeoinformaticsProject\Data\Processed\MODIS_SnowMask_Clean"

    for year in ["2022", "2023"]:
        in_dir  = os.path.join(BASE_IN,  year)
        out_dir = os.path.join(BASE_OUT, year)
        os.makedirs(out_dir, exist_ok=True)

        for fn in sorted(os.listdir(in_dir)):
            if not fn.lower().endswith(".img"):
                continue

            src_fp = os.path.join(in_dir, fn)

            # strip off .img and give it a .tif extension (no "_clean" suffix)
            base = os.path.splitext(fn)[0]      # e.g. "MOD09A1_20220101"
            clean_fn = base + ".tif"            # → "MOD09A1_20220101.tif"

            dst_fp = os.path.join(out_dir, clean_fn)
            print(f"Cleaning {src_fp} → {dst_fp}")
            prepare_modis_mask(src_fp, dst_fp)


Cleaning F:\GeoinformaticsProject\Data\MODIS_SnowMask\2022\neve_20211220.img → F:\GeoinformaticsProject\Data\Processed\MODIS_SnowMask_Clean\2022\neve_20211220.tif
Cleaning F:\GeoinformaticsProject\Data\MODIS_SnowMask\2022\neve_20220101.img → F:\GeoinformaticsProject\Data\Processed\MODIS_SnowMask_Clean\2022\neve_20220101.tif
Cleaning F:\GeoinformaticsProject\Data\MODIS_SnowMask\2022\neve_20220111.img → F:\GeoinformaticsProject\Data\Processed\MODIS_SnowMask_Clean\2022\neve_20220111.tif
Cleaning F:\GeoinformaticsProject\Data\MODIS_SnowMask\2022\neve_20220116.img → F:\GeoinformaticsProject\Data\Processed\MODIS_SnowMask_Clean\2022\neve_20220116.tif
Cleaning F:\GeoinformaticsProject\Data\MODIS_SnowMask\2022\neve_20220124.img → F:\GeoinformaticsProject\Data\Processed\MODIS_SnowMask_Clean\2022\neve_20220124.tif
Cleaning F:\GeoinformaticsProject\Data\MODIS_SnowMask\2022\neve_20220130.img → F:\GeoinformaticsProject\Data\Processed\MODIS_SnowMask_Clean\2022\neve_20220130.tif
Cleaning F:\Geoinforma

In [2]:
import rasterio
import numpy as np
arr = rasterio.open(r"F:\GeoinformaticsProject\Data\MODIS_SnowMask\2023/neve_20230212.img").read(1)
print(np.unique(arr))  # should print [0  1  255]

[1 2]


In [3]:
import rasterio
import numpy as np
arr = rasterio.open(r"F:\GeoinformaticsProject\Data\Processed\MODIS_SnowMask_Clean\2023/neve_20230406.tif").read(1)
print(np.unique(arr))  # should print [0  1  255]

[0 1]


In [1]:
import os
from rasterio.enums import Resampling
from snow_processing import reproject_resample_visualize

input_base       = r"F:\GeoinformaticsProject\Data\Processed\MODIS_SnowMask_Clean"
resampled_output = r"F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\MODIS"
dst_crs          = "EPSG:32632"
dst_res          = (60, 60)

os.makedirs(resampled_output, exist_ok=True)

for year in ["2022", "2023"]:
    year_folder = os.path.join(input_base, year)
    if not os.path.isdir(year_folder):
        continue

    for fname in sorted(os.listdir(year_folder)):
        if not (fname.startswith("neve_") and fname.endswith(".tif")):
            continue

        src_path = os.path.join(year_folder, fname)
        base     = os.path.splitext(fname)[0]
        dst_path = os.path.join(resampled_output, f"{base}.tif")

        if os.path.exists(dst_path):
            print(f"⚡ Skipping existing {base}.tif")
            continue

        reproject_resample_visualize(
            src_path          = src_path,
            dst_path          = dst_path,
            dst_crs           = dst_crs,
            dst_res           = dst_res,
            resampling_method = Resampling.nearest,
            visualize         = False
        )
        print(f"✅ Resampled {base}.tif")


f:\GeoinformaticsProject\Code\Snow\snow_processing.py:47: SyntaxWarning: invalid escape sequence '\M'
  """


✅ Resampled & reprojected saved to: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\MODIS\neve_20211220.tif
✅ Resampled neve_20211220.tif
✅ Resampled & reprojected saved to: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\MODIS\neve_20220101.tif
✅ Resampled neve_20220101.tif
✅ Resampled & reprojected saved to: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\MODIS\neve_20220111.tif
✅ Resampled neve_20220111.tif
✅ Resampled & reprojected saved to: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\MODIS\neve_20220116.tif
✅ Resampled neve_20220116.tif
✅ Resampled & reprojected saved to: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\MODIS\neve_20220124.tif
✅ Resampled neve_20220124.tif
✅ Resampled & reprojected saved to: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\MODIS\neve_20220130.tif
✅ Resampled neve_20220130.tif
✅ Resampled & reprojected saved to: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-RE

In [2]:
import os
from snow_processing import aggregate_weekly

resampled_modis = r"F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\MODIS"
weekly_modis   = r"F:\GeoinformaticsProject\Data\Processed\WEEKLY\MODIS"

print("\n▶️ Aggregating weekly for MODIS")
aggregate_weekly(
    directory     = resampled_modis,
    pattern       = '.tif',      # all binary MODIS .tif files
    output_dir    = weekly_modis,
    method        = 'max',
    threshold     = 0.5,        # keep raw 0/1
    qc_suffix     = '_QC.tif',   # no effect (no QC files)
    skip_existing = True,
    summary       = True,
    parallel      = True,
    max_workers   = 2
)


▶️ Aggregating weekly for MODIS
📦 Aggregating 1 → 2021_W51 (max)
📦 Aggregating 1 → 2021_W52 (max)


f:\GeoinformaticsProject\Code\Snow\snow_processing.py:329: RuntimeWarning: All-NaN slice encountered
  result = np.nanmax(arr, axis=0)


📦 Aggregating 2 → 2022_W02 (max)
📦 Aggregating 2 → 2022_W04 (max)
📦 Aggregating 1 → 2022_W06 (max)
📦 Aggregating 1 → 2022_W07 (max)
📦 Aggregating 1 → 2022_W09 (max)📦 Aggregating 1 → 2022_W08 (max)

📦 Aggregating 1 → 2022_W10 (max)
📦 Aggregating 1 → 2022_W11 (max)
📦 Aggregating 2 → 2022_W12 (max)
📦 Aggregating 2 → 2022_W14 (max)
📦 Aggregating 1 → 2022_W16 (max)📦 Aggregating 1 → 2022_W17 (max)

📦 Aggregating 1 → 2022_W43 (max)📦 Aggregating 1 → 2022_W19 (max)

📦 Aggregating 1 → 2022_W44 (max)📦 Aggregating 1 → 2022_W47 (max)

📦 Aggregating 1 → 2022_W49 (max)
📦 Aggregating 2 → 2022_W50 (max)
📦 Aggregating 1 → 2022_W52 (max)
📦 Aggregating 1 → 2023_W02 (max)
📦 Aggregating 1 → 2023_W04 (max)
📦 Aggregating 2 → 2023_W06 (max)
📦 Aggregating 1 → 2023_W08 (max)
📦 Aggregating 1 → 2023_W09 (max)
📦 Aggregating 1 → 2023_W10 (max)
📦 Aggregating 1 → 2023_W11 (max)
📦 Aggregating 1 → 2023_W13 (max)
📦 Aggregating 1 → 2023_W14 (max)
📦 Aggregating 2 → 2023_W15 (max)
📦 Aggregating 1 → 2023_W17 (max)
📦 Aggregat

['F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2021_W51.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2021_W52.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2022_W02.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2022_W04.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2022_W07.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2022_W06.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2022_W09.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2022_W08.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2022_W11.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2022_W10.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2022_W12.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2022_W14.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\WEEKLY\\MODIS\\2022_W16.tif',
 'F:\\Geoinf

In [3]:
import rasterio
import numpy as np
arr = rasterio.open(r"F:\GeoinformaticsProject\Data\Processed\WEEKLY\MODIS/2022_W07.tif").read(1)
print(np.unique(arr))  # should print [0  1  255]

[  0   1 255]


In [1]:
from snow_processing import resample_reproject_gfsc, aggregate_weekly_gfsc
import rasterio
import numpy as np

# 1) Reproject & binarize daily GFSC
daily_in  = r"F:\GeoinformaticsProject\Data\GFSC\NewGFSC"
daily_out = r"F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\GFSC"
resampled = resample_reproject_gfsc(
    input_dir = daily_in,
    output_dir= daily_out,
    threshold = 20.0
)

# check one day’s unique values
with rasterio.open(resampled[0]) as src:
    print("Daily  →", np.unique(src.read(1)))
# should print: [  0   1 255]

# 2) Aggregate to weekly
weekly_in  = daily_out
weekly_out = r"F:\GeoinformaticsProject\Data\Processed\WEEKLY\GFSC"
weekly = aggregate_weekly_gfsc(
    input_dir  = weekly_in,
    output_dir = weekly_out,
    method     = 'max'
)


f:\GeoinformaticsProject\Code\Snow\snow_processing.py:47: SyntaxWarning: invalid escape sequence '\M'
  """


✅ Daily GFSC → F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\GFSC\20211107.tif (kept 38.0% pixels via QC∈[0, 1, 2, 3])
✅ Daily GFSC → F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\GFSC\20211114.tif (kept 11.3% pixels via QC∈[0, 1, 2, 3])
✅ Daily GFSC → F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\GFSC\20211121.tif (kept 7.8% pixels via QC∈[0, 1, 2, 3])
✅ Daily GFSC → F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\GFSC\20211128.tif (kept 30.1% pixels via QC∈[0, 1, 2, 3])
✅ Daily GFSC → F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\GFSC\20211205.tif (kept 29.9% pixels via QC∈[0, 1, 2, 3])
✅ Daily GFSC → F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\GFSC\20211212.tif (kept 46.3% pixels via QC∈[0, 1, 2, 3])
✅ Daily GFSC → F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\GFSC\20211219.tif (kept 28.0% pixels via QC∈[0, 1, 2, 3])
✅ Daily GFSC → F:\GeoinformaticsProject\Data\Processed\R

f:\GeoinformaticsProject\Code\Snow\snow_processing.py:570: RuntimeWarning: All-NaN slice encountered
  res = np.nanmax(arr3d, axis=0)


✔️ Weekly: processed=51, skipped=0, outputs=51


In [1]:

from snow_processing import process_s2_weekly

input_root  = r"F:\GeoinformaticsProject\Data\S2\S2\SnowMask"
output_dir  = r"F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2"

process_s2_weekly(input_root, output_dir)


f:\GeoinformaticsProject\Code\Snow\snow_processing.py:47: SyntaxWarning: invalid escape sequence '\M'
  """


✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2\2022_W01.tif
✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2\2022_W02.tif
✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2\2022_W03.tif
✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2\2022_W04.tif
✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2\2022_W05.tif
✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2\2022_W06.tif
✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2\2022_W07.tif
✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2\2022_W08.tif
✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2\2022_W09.tif
✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2\2022_W10.tif
✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2\2022_W11.tif
✅ Wrote F:\GeoinformaticsProject\Data\Processed\RESAMP

['F:\\GeoinformaticsProject\\Data\\Processed\\RESAMPLED-REPROJECTED\\S2\\2022_W01.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\RESAMPLED-REPROJECTED\\S2\\2022_W02.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\RESAMPLED-REPROJECTED\\S2\\2022_W03.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\RESAMPLED-REPROJECTED\\S2\\2022_W04.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\RESAMPLED-REPROJECTED\\S2\\2022_W05.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\RESAMPLED-REPROJECTED\\S2\\2022_W06.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\RESAMPLED-REPROJECTED\\S2\\2022_W07.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\RESAMPLED-REPROJECTED\\S2\\2022_W08.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\RESAMPLED-REPROJECTED\\S2\\2022_W09.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\RESAMPLED-REPROJECTED\\S2\\2022_W10.tif',
 'F:\\GeoinformaticsProject\\Data\\Processed\\RESAMPLED-REPROJECTED\\S2\\2022_W11.tif',
 'F:\\GeoinformaticsProject\\Dat

In [1]:
from snow_processing import reproject_s3_weekly

s3_input  = r"F:\GeoinformaticsProject\Data\S3\S3\SnowMask"
s3_output = r"F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3"

outs = reproject_s3_weekly(s3_input, s3_output)
print("Final S3 outputs:")
for o in outs:
    print(" ", o)


f:\GeoinformaticsProject\Code\Snow\snow_processing.py:47: SyntaxWarning: invalid escape sequence '\M'
  """


✅ Wrote weekly composite: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3\2022_W01.tif
✅ Wrote weekly composite: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3\2022_W02.tif
✅ Wrote weekly composite: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3\2022_W03.tif
✅ Wrote weekly composite: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3\2022_W04.tif
✅ Wrote weekly composite: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3\2022_W05.tif
✅ Wrote weekly composite: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3\2022_W06.tif
✅ Wrote weekly composite: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3\2022_W07.tif
✅ Wrote weekly composite: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3\2022_W08.tif
✅ Wrote weekly composite: F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3\2022_W09.tif
✅ Wrote weekly composite: F:\GeoinformaticsProject\Data\Processe

In [2]:
from snow_processing import check_all_products_side_by_side
import pandas as pd

# 1) Point at your weekly‐folder locations:
product_folders = [
    ("MODIS", r"F:\GeoinformaticsProject\Data\Processed\WEEKLY\MODIS", False),
    ("GFSC",  r"F:\GeoinformaticsProject\Data\Processed\WEEKLY\GFSC",  False),   
    ("S2",    r"F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2",    False),
    ("S3",    r"F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3",    False),
]

# 2) Build the presence/absence table
df = check_all_products_side_by_side(product_folders,
                                     start_year=2021,
                                     end_year=2023)

# 3) Display it if you like
pd.set_option("display.max_rows", None)
display(df)

# 4) Pull out the list of weeks where ALL products are present
common_weeks = df.loc[df.Common_Week == "✅", "Week"].tolist()
print("common_weeks=", common_weeks)


🔍 Scanning MODIS → F:\GeoinformaticsProject\Data\Processed\WEEKLY\MODIS
   ➤ Found 33 unique weeks
🔍 Scanning GFSC → F:\GeoinformaticsProject\Data\Processed\WEEKLY\GFSC
   ➤ Found 51 unique weeks
🔍 Scanning S2 → F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2
   ➤ Found 46 unique weeks
🔍 Scanning S3 → F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3
   ➤ Found 39 unique weeks


,Week,MODIS,GFSC,S2,S3,Common_Week
0,2021_W01,❌,❌,❌,❌,
1,2021_W02,❌,❌,❌,❌,
2,2021_W03,❌,❌,❌,❌,
3,2021_W04,❌,❌,❌,❌,
4,2021_W05,❌,❌,❌,❌,
5,2021_W06,❌,❌,❌,❌,
6,2021_W07,❌,❌,❌,❌,
7,2021_W08,❌,❌,❌,❌,
8,2021_W09,❌,❌,❌,❌,
9,2021_W10,❌,❌,❌,❌,


common_weeks= ['2022_W02', '2022_W04', '2022_W06', '2022_W07', '2022_W08', '2022_W09', '2022_W10', '2022_W11', '2022_W12', '2022_W44', '2022_W47', '2022_W49', '2022_W50', '2023_W04', '2023_W06', '2023_W08', '2023_W09', '2023_W10', '2023_W11', '2023_W14', '2023_W15', '2023_W17']


In [2]:
import os
from snow_processing import match_raster_grid
from rasterio.enums import Resampling

# 1) Your reference (MODIS) weekly directory
modis_dir = r"F:\GeoinformaticsProject\Data\Processed\WEEKLY\MODIS"

# 2) Source folders for the other products
src_dirs = {
    "GFSC": r"F:\GeoinformaticsProject\Data\Processed\WEEKLY\GFSC",
    #"S2"  : r"F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S2",
    #"S3"  : r"F:\GeoinformaticsProject\Data\Processed\RESAMPLED-REPROJECTED\S3",
}

# 3) Where the aligned outputs will go
aligned_base = r"F:\GeoinformaticsProject\Data\Processed\ALIGNED"
aligned_dirs = {prod: os.path.join(aligned_base, prod) for prod in src_dirs}
for d in aligned_dirs.values():
    os.makedirs(d, exist_ok=True)

# 4) Your explicitly defined common weeks
common_weeks= [
    '2022_W02', '2022_W04', '2022_W06', '2022_W07', '2022_W08', 
    '2022_W09', '2022_W10', '2022_W11', '2022_W12', '2022_W44', 
    '2022_W47', '2022_W49', '2022_W50', 
    '2023_W04', '2023_W06', '2023_W08', '2023_W09', '2023_W10', 
    '2023_W11', '2023_W14', '2023_W15', '2023_W17']

# 5) Loop and match the grids
for prod, src_folder in src_dirs.items():
    dst_folder = aligned_dirs[prod]
    for wk in common_weeks:
        ref_fp = os.path.join(modis_dir,  f"{wk}.tif")
        tgt_fp = os.path.join(src_folder, f"{wk}.tif")
        out_fp = os.path.join(dst_folder, f"{wk}.tif")

        if not os.path.exists(ref_fp):
            print(f"⚠️  Missing MODIS {wk}, skipping {prod}")
            continue
        if not os.path.exists(tgt_fp):
            print(f"⚠️  Missing {prod} {wk}, skipping")
            continue

        match_raster_grid(
            reference_path    = ref_fp,
            target_path       = tgt_fp,
            output_path       = out_fp,
            resampling_method = Resampling.nearest
        )
        print(f"✅ Aligned {prod} {wk} → {out_fp}")


✅ Reprojected to match grid: F:\GeoinformaticsProject\Data\Processed\ALIGNED\GFSC\2022_W02.tif
✅ Aligned GFSC 2022_W02 → F:\GeoinformaticsProject\Data\Processed\ALIGNED\GFSC\2022_W02.tif
✅ Reprojected to match grid: F:\GeoinformaticsProject\Data\Processed\ALIGNED\GFSC\2022_W04.tif
✅ Aligned GFSC 2022_W04 → F:\GeoinformaticsProject\Data\Processed\ALIGNED\GFSC\2022_W04.tif
✅ Reprojected to match grid: F:\GeoinformaticsProject\Data\Processed\ALIGNED\GFSC\2022_W06.tif
✅ Aligned GFSC 2022_W06 → F:\GeoinformaticsProject\Data\Processed\ALIGNED\GFSC\2022_W06.tif
✅ Reprojected to match grid: F:\GeoinformaticsProject\Data\Processed\ALIGNED\GFSC\2022_W07.tif
✅ Aligned GFSC 2022_W07 → F:\GeoinformaticsProject\Data\Processed\ALIGNED\GFSC\2022_W07.tif
✅ Reprojected to match grid: F:\GeoinformaticsProject\Data\Processed\ALIGNED\GFSC\2022_W08.tif
✅ Aligned GFSC 2022_W08 → F:\GeoinformaticsProject\Data\Processed\ALIGNED\GFSC\2022_W08.tif
✅ Reprojected to match grid: F:\GeoinformaticsProject\Data\Proces

In [3]:
import geopandas as gpd
from snow_processing import clip_weekly_to_roi

# 1) Load & reproject your Lombardy ROI geojson
roi = gpd.read_file("F:/GeoinformaticsProject/Data/Lombardy.geojson")
roi = roi.to_crs("EPSG:32632")
shapes = [feature["geometry"] for feature in roi.__geo_interface__["features"]]

# 2) Point to your already‐aligned weekly folders
weekly_dirs = {
  #"MODIS": r"F:/GeoinformaticsProject/Data/Processed/WEEKLY/MODIS",
  "GFSC" : r"F:/GeoinformaticsProject\Data/Processed/ALIGNED/GFSC",
  #"S2"   : r"F:/GeoinformaticsProject\Data/Processed/ALIGNED/S2",
  #"S3"   : r"F:/GeoinformaticsProject\Data/Processed/ALIGNED/S3",
}


# 3) Only clip the common weeks you care about
common_weeks= [
    '2022_W02', '2022_W04', '2022_W06', '2022_W07', '2022_W08', 
    '2022_W09', '2022_W10', '2022_W11', '2022_W12', '2022_W44', 
    '2022_W47', '2022_W49', '2022_W50', 
    '2023_W04', '2023_W06', '2023_W08', '2023_W09', '2023_W10', 
    '2023_W11', '2023_W14', '2023_W15', '2023_W17']

# 4) Where to write the clipped rasters
clipped_root = r"F:/GeoinformaticsProject/Data/Processed/CLIPPED"

# 5) Run it!
clip_weekly_to_roi(weekly_dirs, shapes, common_weeks, clipped_root)


🏷  Clipped GFSC 2022_W02 → F:/GeoinformaticsProject/Data/Processed/CLIPPED\GFSC\2022_W02.tif
🏷  Clipped GFSC 2022_W04 → F:/GeoinformaticsProject/Data/Processed/CLIPPED\GFSC\2022_W04.tif
🏷  Clipped GFSC 2022_W06 → F:/GeoinformaticsProject/Data/Processed/CLIPPED\GFSC\2022_W06.tif
🏷  Clipped GFSC 2022_W07 → F:/GeoinformaticsProject/Data/Processed/CLIPPED\GFSC\2022_W07.tif
🏷  Clipped GFSC 2022_W08 → F:/GeoinformaticsProject/Data/Processed/CLIPPED\GFSC\2022_W08.tif
🏷  Clipped GFSC 2022_W09 → F:/GeoinformaticsProject/Data/Processed/CLIPPED\GFSC\2022_W09.tif
🏷  Clipped GFSC 2022_W10 → F:/GeoinformaticsProject/Data/Processed/CLIPPED\GFSC\2022_W10.tif
🏷  Clipped GFSC 2022_W11 → F:/GeoinformaticsProject/Data/Processed/CLIPPED\GFSC\2022_W11.tif
🏷  Clipped GFSC 2022_W12 → F:/GeoinformaticsProject/Data/Processed/CLIPPED\GFSC\2022_W12.tif
🏷  Clipped GFSC 2022_W44 → F:/GeoinformaticsProject/Data/Processed/CLIPPED\GFSC\2022_W44.tif
🏷  Clipped GFSC 2022_W47 → F:/GeoinformaticsProject/Data/Processed/CLI